In [46]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
import os
os.getcwd()

'/Users/hty/Desktop/ECE1786/GOTalk/train'

In [100]:
file = open("../data/Jon_version2.txt", "r")
full_texts =[]
texts = []
labels = []
for i in range(910):
    line = file.readline()
    words = word_tokenize(line)
    sentence = TreebankWordDetokenizer().detokenize(words)
    if sentence[-1] == ']':
        sentence = sentence + ' '
    full_texts.append(sentence)
    # texts.append(TreebankWordDetokenizer().detokenize(words[:-1]))
    # labels.append(TreebankWordDetokenizer().detokenize(words[1:]))
    
train, test = train_test_split(full_texts, test_size=0.2, shuffle=False)

In [101]:
file = open('../data/txtfiles/train_v3.txt', 'w')
for i in train:
    print(i)
    file.write(i)
file.close()

file = open('../data/txtfiles/test_v3.txt', 'w')
for i in test:
    file.write(i)
file.close()

There were times—not many, but a few—when Jon Snow was glad he was a bastard . As he filled his wine cup once more from a passing flagon, it struck him that this might be one of them.
He settled back in his place on the bench among the younger squires and drank . The sweet, fruity taste of summerwine filled his mouth and brought a smile to his lips.
The Great Hall of Winterfell was hazy with smoke and heavy with the smell of roasted meat and fresh-baked bread . Its grey stone walls were draped with banners . White, gold, crimson: the direwolf of Stark, Baratheon ’ s crowned stag, the lion of Lannister . A singer was playing the high harp and reciting a ballad, but down at this end of the hall his voice could scarcely be heard above the roar of the fire, the clangor of pewter plates and cups, and the low mutter of a hundred drunken conversations.
It was the fourth hour of the welcoming feast laid for the king . Jon ’ s brothers and sisters had been seated with the royal children, beneat

In [67]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("HScomcom/gpt2-game-of-thrones")
train_path = '../data/txtfiles/train.txt'
test_path = '../data/txtfiles/test.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=20,
          )
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=20,
          )   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)



loading configuration file config.json from cache at /Users/hty/.cache/huggingface/hub/models--HScomcom--gpt2-game-of-thrones/snapshots/481371376135f570a8c1a4681ccddede9f305acb/config.json
Model config GPT2Config {
  "_name_or_path": "HScomcom/gpt2-game-of-thrones",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
 

In [58]:
tokenizer

PreTrainedTokenizerFast(name_or_path='HScomcom/gpt2-game-of-thrones', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [43]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("HScomcom/gpt2-game-of-thrones")


training_args = TrainingArguments(
    output_dir="../model", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,

)

trainer.train()
trainer.save_model()

loading configuration file config.json from cache at /Users/hty/.cache/huggingface/hub/models--HScomcom--gpt2-game-of-thrones/snapshots/481371376135f570a8c1a4681ccddede9f305acb/config.json
Model config GPT2Config {
  "_name_or_path": "HScomcom/gpt2-game-of-thrones",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
 

Downloading:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /Users/hty/.cache/huggingface/hub/models--HScomcom--gpt2-game-of-thrones/snapshots/481371376135f570a8c1a4681ccddede9f305acb/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at HScomcom/gpt2-game-of-thrones.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 2614
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & a

  0%|          | 0/7842 [00:00<?, ?it/s]

KeyboardInterrupt: 